<a href="https://colab.research.google.com/github/albachiarabellaroba/HW3---Group-16/blob/main/Copia_di_main_Albachiara.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Homework 3 - Places of the world**

In [29]:
from bs4 import BeautifulSoup
import bs4
import requests
import os.path
import lxml
import re
import codecs
import csv
import nltk
from datetime import datetime
from tqdm import tqdm
import time
import pandas as pd

In [30]:
from google.colab import drive
drive.mount('/content/drive/') 

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


## 1. Data collection

### 1.1. Get the list of places

In [15]:
linklist = open('/content/drive/MyDrive/ADM_HW3/linklist.txt', 'w')

for i in tqdm(range(1, 401)):
    link = f'https://www.atlasobscura.com/places?page={i}&sort=likes_count'
    cont = requests.get(link)
    soup = BeautifulSoup(cont.content, features='lxml')
    for a in soup.find_all('a', class_='content-card content-card-place'):
        linklist.write(a.get('href')+'\n')
        
linklist.close()

100%|██████████| 400/400 [02:05<00:00,  3.18it/s]


In [19]:
file = open('/content/drive/MyDrive/ADM_HW3/linklist.txt', 'r')
count=0
for i in file:
  if i != '\n':
    count += 1
file.close
print(count)

7200


### 1.2. Crawl places

In [20]:
def download(start, finish):
    header = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
    for i in tqdm(range(start, finish)):
      cart = '/content/drive/MyDrive/ADM_HW3/Html_page/page_{}'.format(i)
      os.makedirs(cart)
      
      f = open('/content/drive/MyDrive/ADM_HW3/linklist.txt', 'r', encoding='utf8')
      l = f.readlines()[(i-1)*18:(i)*18]
      l = [k.rstrip() for k in l]
      f.close

      m = 1+18*(i-1)

      for link in l:
        link = 'https://www.atlasobscura.com' + link
        html = requests.get(link, headers = header)
        if html.status_code != 200:
          time.sleep(200)
          html = requests.get(link, headers = header)


        name = '{}/{}.html'.format(cart, m)
        text = open(name, 'w', encoding='utf8')
        text.write(html.text)
        text.close
        m=m+1


In [21]:
download(301, 401)

100%|██████████| 100/100 [26:07<00:00, 15.67s/it]


### 1.3 Parse downloaded pages

In [31]:
placeName=[]
placeTags=[]
numPeopleVisited=[]
numPeopleWant=[]
placeDesc=[]
placeShortDesc=[]
placeNearby=[]
placeAddress=[]
placeAlt=[]
placeLong=[]
placeEditors=[]
placePubDate=[]
placeRelatedLists=[]
placeRelatedPlaces=[]
placeURL=[]

In [39]:
 for i in tqdm(range(1,401)):
   files = os.listdir('/content/drive/MyDrive/ADM_HW3/Html_page/page_'+str(i))
   for k in range(((18*(int(i)-1))+1), (18*int(i)+1)):
     file = open('/content/drive/MyDrive/ADM_HW3/Html_page/page_'+str(i)+'/'+str(k)+'.html', 'r', encoding='utf8')
     soup=bs4.BeautifulSoup(file, 'lxml')

     try:
       Name=soup.find_all('h1', {'class':'DDPage__header-title'})['content']
       placeName.append(Name)
     except:
         placeName.append('')
         
     try:
       tag=soup.find_all('a', {'class':'itemTags__link js-item-tags-link'})
       tag=[i.get('href').split('/categories/')[1] for i in tag]
       placeTags.append(tag)
     except:
       placeTags.append('')
       
     try:
       visitors=soup.find_all('div', {'class':'title-md item-action-count'})[0].text.strip()
       numPeopleVisited.append(visitors)
     except:
       numPeopleVisited.append('')
    
     try:
       want=soup.find_all('div', {'class':'title-md item-action-count'})[1].text.strip()
       numPeopleWant.append(want)
     except:
       numPeopleWant.append('')

     try:
       desc=soup.find_all('div', {'class':'DDP__body-copy'}).text.strip()
       placeDesc.append(desc)
     except:
       placeDesc.append('')

     try:
       shdesc=soup.find_all('h3', {'class':'DDPage__header-dek'})['content']
       placeShortDesc.append(shdesc)
     except:
       placeShortDesc.append('')

     try:
       nearby=soup.find_all('div', {'class':'DDPageSiderailRecirc__item-title'})
       nearby=[i.get_text() for i in nearby]
       placeNearby.append(nearby)
     except:
       placeNearby.append('')

     try:
       address=soup.find_all('address', {'class':'DDPageSiderail__address'})[0]
       address=address.find('div').get_text(separator=' ').split('\n')[0]
       placeAddress.append(address)
     except:
       placeAddress.append('')

     try:
       palt, plong=soup.find_all('div', {'class':'DDPageSiderail__coordinates js-copy-coordinates'})
       placeAlt.append(palt)
       placeLong.append(plong)
     except:
       placeAddress.append('')

     try:
       editors=soup.find_all('div', {'class':'ugc-editor-icons'})
       editors=[i.findChildren('a')[0].get('href') for i in editors]
       editors=[k.split('/users/')[1] for k in editors]
       placeEditors.append(editors)
     except:
       placeEditors.append('')

     try:
       date=soup.find_all('div', {'class':'DDPContributor__name'}).text.strip()
       placePubDate.append('')
     except:
       placePubDate.append('')

     try:
       rellist=soup.find_all('div', {'data-gtm-template':'DDP Footer Recirc Lists'})[0]
       rellist=[i.text.strip() for i in rellist.find_all('h3')]
       placeRelatedLists.append(rellist)
     except:
       placeRelatedLists.append('')

     try:
       relplaces=soup.find_all('div', {'class':'CardRecircSection__title'})
       relplaces=[i for i in relplaces if i.text == 'Related Places'][0]
       relplaces=relplaces.find_all_next('div', {'class':'Card__action-btns vue-js-been-there-everywhere-place'})
       relplace=[k.get('data-place-title') for k in relplace]
       placeRelatedPlaces.append(relplaces)
     except:
       placeRelatedPlaces.append('')

     try:
       url=soup.find('meta', attrs={'property':'og:url'})['content']
       placeURL.append(url)
     except:
       placeURL.append('')

 10%|█         | 40/400 [01:40<15:07,  2.52s/it]


KeyboardInterrupt: ignored

In [14]:
for i in tqdm(range(1,401)):
  files = os.listdir('/content/drive/MyDrive/ADM_HW3/Html_page/page_'+str(i))
        for k in range(((18*(int(i)-1))+1), (18*int(i)+1)):
          file = open('/content/drive/MyDrive/ADM_HW3/Html_page/page_'+str(i)+'/'+str(k)+'.html', 'r', encoding='utf8')
          soup=bs4.BeautifulSoup(file, 'lxml')

In [ ]:
  try:
      title=soup.find_all('h1', {'class':'DDPage__header-title'})['contents']
      Name.append(title)
  except:
      Name.append('')

  try:
      title=soup.find_all('h1', {'class':'DDPage__header-title'})['contents']
      Name.append(title)
  except:
      Name.append('')
  
  try:
      title=soup.find_all('h1', {'class':'DDPage__header-title'})['contents']
      Name.append(title)
  except:
      Name.append('')
  
  try:
      title=soup.find_all('h1', {'class':'DDPage__header-title'})['contents']
      Name.append(title)
  except:
      Name.append('')
  
  try:
      title=soup.find_all('h1', {'class':'DDPage__header-title'})['contents']
      Name.append(title)
  except:
      Name.append('')
        #inseriscri funzione per salvare file tsv

    

In [ ]:
for i in tqdm(range(1,401)):  #numero castella pagina     400 cart ognuna 18pg
      for k in range(((18*(int(i)-1))+1), (18*int(i)+1)):   # k= 1-18 , 19-36 ....
        file = open('/content/drive/MyDrive/ADM_HW3/Html_page/page_'+str(i)+'/'+str(k)+'.html', 'r', encoding='utf8')
        placeName(file)
        placeTags(file)


In [13]:
#come salvare file tsv

with open("percorso") as fp:
  csv.writer(fp, delimiter = "\t").writerow([placeTag, placeName....])

  8%|▊         | 24/300 [00:51<09:50,  2.14s/it]


KeyboardInterrupt: ignored

In [ ]:
print(placeName)
print(len(placeName))

['77 Water Street', "World's Largest Cowboy Boots ", 'The Lost Town of Newtown Jerpoint (Jerpoint Park)', 'American Museum of the House Cat', 'Titanic Memorial', 'Delaware Water Gap National Recreation Area', 'Kakslauttanen Arctic Resort', 'Prehistoric Gardens', 'The Recumbent Effigy of Victor Noir', "F. Scott Fitzgerald's Boarding House", 'The Plant', 'Thistle Ghost Town', 'Salem Village Parsonage', 'East Potomac Park Mini Golf', 'Interference Archive', "Goatman's Bridge", 'Nevadaville Ghost Town', 'Umschreibung', 'Buena Vista Park Tombstones', 'The DNA Discovery Center', 'Tenshou-Kyousha Shrine Mermaid Mummy', 'Paternoster Lift', 'Vanadu Art House', 'Sparky Park', 'Preston Castle', 'Miniature Garden of Whimsy ', 'The Faerie Playhouse', 'Mount Elbert', 'Saalfeld Fairy Grottoes', 'Pet Sematary Filming Sites', 'Blue Whale of Catoosa', 'Museum of Health and Medical Science', 'Beth Haim Cemetery', 'Tree Climbing Planet', 'Berliner Medizinhistorisches Museum', 'Gates of Hell Sculpture', "S

In [ ]:
dati={'Place Name':[placeName], 'Place Tags':[placeTags], 'Visitors':[numPeopleVisited], 'Want to visit':[numPeopleWant],
      'Description':[placeDesc], 'Short Description':[placeShortDesc], 'Nearby Places':[placeNearby], 'Address':[placeAddress], 
      'Altitude':[placealt], 'Longitude':[placeLong], 'Editors':[placeEditors], 'Publishing Date':[placePubDate],
      'Included':[placeRelatedLists], 'Related':[placeRelatedPlaces], 'URL':[placeURL]}
features=pd.DataFrame(data=dati, columns=['Place Name', 'Place Tags', 'Visitors', 'Want to visit', 'Description', 'Short Dscription',
                                          'Nearby Places', 'Address', 'Altitude', 'Longitude', 'Editors', 'Publishing Date', 
                                          'Included', 'Related', 'URL'])

os.makedirs('/content/drive/MyDrive/ADM_HW3/tsv_file')
placesfeatures = features.to_csv('content/drive/MyDrive/ADM_HW3/features.tsv', sep='\t', index=False)

## 2. Search Engine

In [ ]:
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

In [ ]:
nltk.downloads('stopwords')
nltk.download('punkt')

In [ ]:
stopwords = set(stopwords.words('english'))

for line in placesfeatures['Description']:
  tokens=nlkt.word_tokenize(line, language='english')
  


In [ ]:
# usa freqdist per contare le frequenze delle parole

### 2.1.1 Create your index!

### 2.1.2 Execute the query

### 2.2 Conjunctive query & Ranking score

### 2.2.1 Inverted index

### 2.2.2 Execute the query

## 3. Define a new score!

## 4. Visualizing the most relevant places

## 5. BONUS: More complex search engine

## 6. Command line question

## 7. Theoretical question